---
title: "Assignment 3-2"
author: "Sangwon Ju, SNU GSPA"
date: 'NOV/27/2022'
format: 
    html:
        code-overflow: wrap
        code-fold: true
categories:
   "Machine Learning & Deep Learning for Data Science (2022 Fall)" 
image: image.png
---

2NN and Fully Connected Layers (Score: 107/100)

# Local Setup

In [1]:
import os
# get current path
print(os.getcwd())
# change path
os.chdir("E:/OneDrive - SNU/(B) 대학원/수업/2022 2학기/데이터사이언스를위한머신러닝과딥러닝/과제3")
print(os.getcwd())

e:\OneDrive - SNU\(B) 대학원\수업\2022 2학기\데이터사이언스를위한머신러닝과딥러닝\과제3
E:\OneDrive - SNU\(B) 대학원\수업\2022 2학기\데이터사이언스를위한머신러닝과딥러닝\과제3


# Import Modules

In [2]:
import copy
import numpy as np
import matplotlib.pyplot as plt
from mnist.data_utils import load_data

# Utils

In [3]:
def tanh(z):
    """
    Implement the tanh activation function.
    The method takes the input z and returns the output of the function.

    Question (a)

    """
    ##### YOUR CODE #####
    tanh = lambda x: (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
    result = tanh(z)
    return result   
    ##################### 

def softmax(X):
    """
    Implement the softmax function.
    The method takes the input X and returns the output of the function.

    Question (a)

    """
    ##### YOUR CODE #####
    elements = np.exp(X-np.amax(X,axis=1,keepdims=True))
    sums = np.sum(elements, axis=1, keepdims=True)
    return elements/sums
    #####################


def load_batch(X, Y, batch_size, shuffle=True):
    """
    Generates batches with the remainder dropped.

    Do NOT modify this function
    """
    if shuffle:
        permutation = np.random.permutation(X.shape[0])
        X = X[permutation, :]
        Y = Y[permutation, :]
    num_steps = int(X.shape[0])//batch_size
    step = 0
    while step<num_steps:
        X_batch = X[batch_size*step:batch_size*(step+1)]
        Y_batch = Y[batch_size*step:batch_size*(step+1)]
        step+=1
        yield X_batch, Y_batch

# 2-Layer Neural Network

In [4]:
class TwoLayerNN:
    """ a neural network with 2 layers """

    def __init__(self, input_dim, num_hiddens, num_classes):
        """
        Do NOT modify this function.
        """
        self.input_dim = input_dim
        self.num_hiddens = num_hiddens
        self.num_classes = num_classes
        self.params = self.initialize_parameters(input_dim, num_hiddens, num_classes)

    def initialize_parameters(self, input_dim, num_hiddens, num_classes):
        """
        initializes parameters with Xavier Initialization.

        Question (b)
        - refer to https://paperswithcode.com/method/xavier-initialization for Xavier initialization 
        
        Inputs
        - input_dim
        - num_hiddens
        - num_classes
        Returns
        - params: a dictionary with the initialized parameters.
        """
        params = {}
        ##### YOUR CODE #####
        params["W1"] = np.random.uniform(low=-1/(np.sqrt(num_hiddens)), high=1/(np.sqrt(num_hiddens)), size=(input_dim, num_hiddens))
        params["b1"] = np.zeros(num_hiddens)
        params["W2"] = np.random.uniform(low=-1/(np.sqrt(num_classes)), high=1/(np.sqrt(num_classes)), size=(num_hiddens, num_classes))
        params["b2"] = np.zeros(num_classes)
        #####################
        return params

    def forward(self, X):
        """
        Define and perform the feed forward step of a two-layer neural network.
        Specifically, the network structue is given by

          y = softmax(tanh(X W1 + b1) W2 + b2)

        where X is the input matrix of shape (N, D), y is the class distribution matrix
        of shape (N, C), N is the number of examples (either the entire dataset or
        a mini-batch), D is the feature dimensionality, and C is the number of classes.

        Question (c)
        - ff_dict will be used to run backpropagation in backward method.

        Inputs
        - X: the input matrix of shape (N, D)

        Returns
        - y: the output of the model
        - ff_dict: a dictionary with all the fully connected units and activations.
        """
        ff_dict = {}
        ##### YOUR CODE #####
        params = self.params
        w1 = params["W1"] 
        b1 = params["b1"] 
        w2 = params["W2"] 
        b2 = params["b2"] 

        R1 = np.dot(X, w1) + b1 # N * H
        Act1 = tanh(R1) # N * H
        R2 = np.dot(Act1, w2) + b2 # N * C 
        Act2 = softmax(R2) # N * C

        ff_dict['Affine1'] =  R1
        ff_dict['Tanh'] = Act1
        ff_dict['Affine2'] = R2
        ff_dict['Softmax'] = Act2
        y = Act2
        #####################
        return y, ff_dict

    def backward(self, X, Y, ff_dict):
        """
        Performs backpropagation over the two-layer neural network, and returns
        a dictionary of gradients of all model parameters.

        Question (d)

        Inputs:
         - X: the input matrix of shape (B, D), where B is the number of examples
              in a mini-batch, D is the feature dimensionality.
         - Y: the matrix of one-hot encoded ground truth classes of shape (B, C),
              where B is the number of examples in a mini-batch, C is the number
              of classes.
         - ff_dict: the dictionary containing all the fully connected units and
              activations.

        Returns:
         - grads: a dictionary containing the gradients of corresponding weights and biases.
        """
        grads = {}
        ##### YOUR CODE #####
        batch_size = X.shape[0]
        Act1 = ff_dict['Affine1'] 
        z1 = ff_dict['Tanh']
        z2 = ff_dict['Softmax']
        w2 = self.params["W2"]

        dz2 = (z2-Y) / batch_size  # normalization & entropy diff
        grads["dW2"] = np.dot(z1.T, dz2)
        grads["db2"] = np.sum(dz2, axis = 0)
    
        dz1 = (1-tanh(Act1)) * (1+tanh(Act1)) * np.dot(dz2, w2.T) # tanh diff
        grads["dW1"] = np.dot(X.T, dz1)
        grads["db1"] = np.sum(dz1, axis = 0)
        #####################
        return grads


    def compute_loss(self, Y, Y_hat):
        """
        Computes cross entropy loss.

        Do NOT modify this function.

        Inputs
            Y:
            Y_hat:
        Returns
            loss:
        """
        loss = -(1/Y.shape[0]) * np.sum(np.multiply(Y, np.log(Y_hat)))
        return loss

    def train(self, X, Y, X_val, Y_val, lr, n_epochs, batch_size, log_interval=1):
        """
        Runs mini-batch gradient descent.

        Do NOT Modify this method.

        Inputs
        - X
        - Y
        - X_val
        - Y_Val
        - lr
        - n_epochs
        - batch_size
        - log_interval
        """
        for epoch in range(n_epochs):
            for X_batch, Y_batch in load_batch(X, Y, batch_size):
                self.train_step(X_batch, Y_batch, batch_size, lr)
            if epoch % log_interval==0:
                Y_hat, ff_dict = self.forward(X)
                train_loss = self.compute_loss(Y, Y_hat)
                train_acc = self.evaluate(Y, Y_hat)
                Y_hat, ff_dict = self.forward(X_val)
                valid_loss = self.compute_loss(Y_val, Y_hat)
                valid_acc = self.evaluate(Y_val, Y_hat)
                print('epoch {:02} - train loss/acc: {:.3f} {:.3f}, valid loss/acc: {:.3f} {:.3f}'.\
                      format(epoch, train_loss, train_acc, valid_loss, valid_acc))

    def train_step(self, X_batch, Y_batch, batch_size, lr):
        """
        Updates the parameters using gradient descent.

        Do NOT Modify this method.

        Inputs
        - X_batch
        - Y_batch
        - batch_size
        - lr
        """
        _, ff_dict = self.forward(X_batch)
        grads = self.backward(X_batch, Y_batch, ff_dict)
        self.params["W1"] -= lr * grads["dW1"]/batch_size
        self.params["b1"] -= lr * grads["db1"]/batch_size
        self.params["W2"] -= lr * grads["dW2"]/batch_size
        self.params["b2"] -= lr * grads["db2"]/batch_size

    def evaluate(self, Y, Y_hat):
        """
        Computes classification accuracy.
        
        Do NOT modify this function

        Inputs
        - Y: A numpy array of shape (N, C) containing the softmax outputs,
             where C is the number of classes.
        - Y_hat: A numpy array of shape (N, C) containing the one-hot encoded labels,
             where C is the number of classes.

        Returns
            accuracy: the classification accuracy in float
        """        
        classes_pred = np.argmax(Y_hat, axis=1)
        classes_gt = np.argmax(Y, axis=1)
        accuracy = float(np.sum(classes_pred==classes_gt)) / Y.shape[0]
        return accuracy

# Load MNIST

In [5]:
X_train, Y_train, X_test, Y_test = load_data()

idxs = np.arange(len(X_train))
np.random.shuffle(idxs)
split_idx = int(np.ceil(len(idxs)*0.8))
X_valid, Y_valid = X_train[idxs[split_idx:]], Y_train[idxs[split_idx:]]
X_train, Y_train = X_train[idxs[:split_idx]], Y_train[idxs[:split_idx]]
print()
print('Set validation data aside')
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', Y_train.shape)
print('Validation data shape: ', X_valid.shape)
print('Validation labels shape: ', Y_valid.shape)

MNIST data loaded:
Training data shape: (60000, 784)
Training labels shape: (60000, 10)
Test data shape: (10000, 784)
Test labels shape: (10000, 10)

Set validation data aside
Training data shape:  (48000, 784)
Training labels shape:  (48000, 10)
Validation data shape:  (12000, 784)
Validation labels shape:  (12000, 10)


# Training & Evaluation

In [6]:
### 
# Question (e)
# Tune the hyperparameters with validation data, 
# and print the results by running the lines below.
###

In [7]:
# model instantiation
model = TwoLayerNN(input_dim=784, num_hiddens=80, num_classes=10)

In [8]:
# train the model
lr, n_epochs, batch_size = 3, 40, 128
model.train(X_train, Y_train, X_valid, Y_valid, lr, n_epochs, batch_size)

epoch 00 - train loss/acc: 0.512 0.867, valid loss/acc: 0.517 0.866
epoch 01 - train loss/acc: 0.406 0.888, valid loss/acc: 0.411 0.888
epoch 02 - train loss/acc: 0.361 0.899, valid loss/acc: 0.366 0.899
epoch 03 - train loss/acc: 0.333 0.906, valid loss/acc: 0.340 0.904
epoch 04 - train loss/acc: 0.314 0.911, valid loss/acc: 0.321 0.909
epoch 05 - train loss/acc: 0.299 0.915, valid loss/acc: 0.307 0.913
epoch 06 - train loss/acc: 0.286 0.918, valid loss/acc: 0.295 0.917
epoch 07 - train loss/acc: 0.275 0.921, valid loss/acc: 0.285 0.920
epoch 08 - train loss/acc: 0.266 0.925, valid loss/acc: 0.277 0.922
epoch 09 - train loss/acc: 0.257 0.927, valid loss/acc: 0.268 0.924
epoch 10 - train loss/acc: 0.249 0.930, valid loss/acc: 0.261 0.927
epoch 11 - train loss/acc: 0.241 0.932, valid loss/acc: 0.254 0.928
epoch 12 - train loss/acc: 0.234 0.935, valid loss/acc: 0.248 0.930
epoch 13 - train loss/acc: 0.228 0.937, valid loss/acc: 0.242 0.931
epoch 14 - train loss/acc: 0.222 0.938, valid lo

In [9]:
# evalute the model on test data
Y_hat, _ = model.forward(X_test)
test_loss = model.compute_loss(Y_test, Y_hat)
test_acc = model.evaluate(Y_test, Y_hat)
print("Final test loss = {:.3f}, acc = {:.3f}".format(test_loss, test_acc))

Final test loss = 0.149, acc = 0.956


# Extra Credit (Optional)

In [32]:
######### define relu #############
class relu:
    def fw(x):
        z=x.copy()
        fwd = lambda k: np.maximum(0, k)
        result = np.vectorize(fwd)(z)
        return result
    def bw(x):
        z=x.copy()
        bwd = lambda k : 0 if k<=0 else 1
        result = np.vectorize(bwd)(z)
        return result        
###################################

def initialize_parameters(self, input_dim, num_hiddens, num_classes):
    """
    initializes parameters with He Initialization.

    Question (f)
    - refer to https://paperswithcode.com/method/he-initialization for He initialization 
    
    Inputs
    - input_dim
    - num_hiddens
    - num_classes
    Returns
    - params: a dictionary with the initialized parameters.
    """

    params = {}
    ##### YOUR CODE #####
    params["W1"] = np.random.uniform(low=-1/(np.sqrt(num_hiddens)), high=1/(np.sqrt(num_hiddens)), size=(input_dim, num_hiddens))
    params["b1"] = np.zeros(num_hiddens)
    params["W2"] = np.random.uniform(low=-1/(np.sqrt(num_classes)), high=1/(np.sqrt(num_classes)), size=(num_hiddens, num_classes))
    params["b2"] = np.zeros(num_classes)
    #####################
    return params

def forward_relu(self, X):
    """
    Defines and performs the feed forward step of a two-layer neural network.
    Specifically, the network structue is given by

        y = softmax(relu(X W1 + b1) W2 + b2)

    where X is the input matrix of shape (N, D), y is the class distribution matrix
    of shape (N, C), N is the number of examples (either the entire dataset or
    a mini-batch), D is the feature dimensionality, and C is the number of classes.

    Question (f)

    Inputs
        X: the input matrix of shape (N, D)

    Returns
        y: the output of the model
        ff_dict: a dictionary containing all the fully connected units and activations.
    """

    ff_dict = {}        
    ##### YOUR CODE #####
    params = self.params
    w1 = params["W1"] 
    b1 = params["b1"] 
    w2 = params["W2"] 
    b2 = params["b2"] 

    R1 = np.dot(X, w1) + b1 # N * H
    Act1 = relu.fw(R1) # N * H
    R2 = np.dot(Act1, w2) + b2 # N * C 
    Act2 = softmax(R2) # N * C

    ff_dict['Affine1'] =  R1
    ff_dict['ReLU'] = Act1
    ff_dict['Affine2'] = R2
    ff_dict['Softmax'] = Act2
    y = Act2
    #####################
    return y, ff_dict

def backward_relu(self, X, Y, ff_dict):
    """
    Performs backpropagation over the two-layer neural network, and returns
    a dictionary of gradients of all model parameters.

    Question (f)

    Inputs:
        - X: the input matrix of shape (B, D), where B is the number of examples
            in a mini-batch, D is the feature dimensionality.
        - Y: the matrix of one-hot encoded ground truth classes of shape (B, C),
            where B is the number of examples in a mini-batch, C is the number
            of classes.
        - ff_dict: the dictionary containing all the fully connected units and
            activations.

    Returns:
        - grads: a dictionary containing the gradients of corresponding weights
            and biases.
    """

    grads = {}
    ##### YOUR CODE #####
    batch_size = X.shape[0]
    Act1 = ff_dict['Affine1'] 
    z1 = ff_dict['ReLU']
    z2 = ff_dict['Softmax']
    w2 = self.params["W2"]

    dz2 = (z2-Y) / batch_size  # normalization & entropy diff
    grads["dW2"] = np.dot(z1.T, dz2)
    grads["db2"] = np.sum(dz2, axis = 0)
    
    dz1 = relu.bw(Act1) * np.dot(dz2, w2.T) # relu diff
    grads["dW1"] = np.dot(X.T, dz1)
    grads["db1"] = np.sum(dz1, axis = 0)
    #####################
    return grads

TwoLayerNNRelu = copy.copy(TwoLayerNN)
TwoLayerNNRelu.initialize_parameters = initialize_parameters
TwoLayerNNRelu.forward = forward_relu
TwoLayerNNRelu.backward = backward_relu

In [11]:
### 
# Question (f)
# Tune the hyperparameters with validation data,
# and print the results by running the lines below.
###

In [33]:
# model instantiation
model_relu = TwoLayerNNRelu(input_dim=784, num_hiddens=80, num_classes=10)

In [34]:
# train the model
lr, n_epochs, batch_size = 2.5, 40, 128
history = model_relu.train(X_train, Y_train, X_valid, Y_valid, lr, n_epochs, batch_size)

epoch 00 - train loss/acc: 0.511 0.868, valid loss/acc: 0.517 0.866
epoch 01 - train loss/acc: 0.399 0.890, valid loss/acc: 0.408 0.886
epoch 02 - train loss/acc: 0.353 0.901, valid loss/acc: 0.361 0.897
epoch 03 - train loss/acc: 0.325 0.909, valid loss/acc: 0.333 0.905
epoch 04 - train loss/acc: 0.306 0.913, valid loss/acc: 0.314 0.909
epoch 05 - train loss/acc: 0.290 0.919, valid loss/acc: 0.299 0.915
epoch 06 - train loss/acc: 0.277 0.922, valid loss/acc: 0.286 0.918
epoch 07 - train loss/acc: 0.264 0.926, valid loss/acc: 0.274 0.921
epoch 08 - train loss/acc: 0.254 0.929, valid loss/acc: 0.265 0.923
epoch 09 - train loss/acc: 0.245 0.932, valid loss/acc: 0.256 0.926
epoch 10 - train loss/acc: 0.238 0.933, valid loss/acc: 0.250 0.927
epoch 11 - train loss/acc: 0.230 0.937, valid loss/acc: 0.241 0.930
epoch 12 - train loss/acc: 0.223 0.938, valid loss/acc: 0.234 0.932
epoch 13 - train loss/acc: 0.216 0.940, valid loss/acc: 0.229 0.933
epoch 14 - train loss/acc: 0.211 0.942, valid lo

In [35]:
Y_hat, _ = model_relu.forward(X_test)
test_loss = model_relu.compute_loss(Y_test, Y_hat)
test_acc = model_relu.evaluate(Y_test, Y_hat)
print("Final test loss = {:.3f}, acc = {:.3f}".format(test_loss, test_acc))

Final test loss = 0.142, acc = 0.960
